NLPAUG installation

In [ ]:
!pip install numpy requests nlpaug
!pip install nltk>=3.4.5
!pip install simpletransformers>=0.61.10

Others dependencies installation (pandas)

In [ ]:
!pip install pandas

Mounting google drive folder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Dependencies import

In [ ]:
import os
import pandas
import shutil

Predefining importation methods

In [ ]:
def augment(value, augmenter):
  if augmenter['datatype'] == float or augmenter['datatype'] == int:
    try:
      float(value)
      return augmenter['method'].augment(value)[0]
    except:
      return value
  else:
    if value != 'nan':
      return augmenter['method'].augment(value)[0]
    else:
      return value

Constants defining

In [ ]:
root_annotation_folder =  "/content/drive/MyDrive/ORKG extraction - Aug/annotations"


Augmeneters defining

Each augmenter is defined by


* A prefix, which serves to name the folder containing the files resulting from the augmentation
* The proper augmentation method
* The datatype on which the method must be ran

In [ ]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
os.environ["MODEL_DIR"] = '../model'

from nlpaug.util import Action

augmenters = [
    {'prefix': 'ocr_1', 'method': nac.OcrAug(), 'datatype': str},
    {'prefix': 'ocr_2', 'method': nac.OcrAug(), 'datatype': str},
    {'prefix': 'key_1', 'method': nac.KeyboardAug(), 'datatype': str},
    {'prefix': 'key_2', 'method': nac.KeyboardAug(), 'datatype': str},
    {'prefix': 'key_3', 'method': nac.KeyboardAug(), 'datatype': str},
    {'prefix': 'rnd_ins', 'method': nac.RandomCharAug(action="insert"), 'datatype': str},
    {'prefix': 'rnd_sub', 'method': nac.RandomCharAug(action="substitute"), 'datatype': str},
    {'prefix': 'rnd_swp', 'method': nac.RandomCharAug(action="swap"), 'datatype': str},
    {'prefix': 'rnd_del', 'method': nac.RandomCharAug(action="delete"), 'datatype': str},
    {'prefix': 'wrd', 'method': naw.SpellingAug(), 'datatype': str},
        ]

Augmented dataframe saving method

In [ ]:
def save_augmented_table(dataframe, prefix, primary_folder):
  augmented_folder_path = f"{primary_folder}_{prefix}"
  try:
    os.mkdir(augmented_folder_path)
  except:
    ...

  shutil.copy(f"{primary_folder}/cea.csv",f"{augmented_folder_path}/cea.csv")
  shutil.copy(f"{primary_folder}/cta.csv",f"{augmented_folder_path}/cta.csv")
  shutil.copy(f"{primary_folder}/cpa.csv",f"{augmented_folder_path}/cpa.csv")
  shutil.copy(f"{primary_folder}/ttd.csv",f"{augmented_folder_path}/ttd.csv")

  dataframe.to_csv(f"{augmented_folder_path}/table.csv", index=False)
  ...

Main logic

In [ ]:
def augmentation_process():

  annotation_folders_paths = [os.path.join(root_annotation_folder, folder_path) for folder_path in os.listdir(root_annotation_folder)]
  for annotation_folder in annotation_folders_paths:
    sub_files_list = os.listdir(annotation_folder)
    if 'table.csv' in sub_files_list:
      table_path = os.path.join(annotation_folder, 'table.csv')
      dataframe = pandas.read_csv(table_path, header=None)
      row_nb = dataframe.shape[0]
      col_nb = dataframe.shape[1]
      for augmenter in augmenters:
        augmented_dataframe = pandas.DataFrame()
        for j in range(0, col_nb): # j is the column index
          new_line = []
          # Collecting data of the same column
          for i in range(0, row_nb): # i is the row index
            augmented_cell = augment(str(dataframe.iloc[i,j]), augmenter)
            new_line.append(augmented_cell)
          new_line = pandas.DataFrame(new_line)
          augmented_dataframe = pandas.concat([augmented_dataframe, new_line], axis=1) # axis = 1 means add data as a new column
        save_augmented_table(augmented_dataframe, augmenter['prefix'], annotation_folder)

Main

In [ ]:
augmentation_process()